<a href="https://colab.research.google.com/github/abdirimoof/ML/blob/main/3amaliyRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

maktab so‘zi uchun ma’lumotlarni tayyorlang.





"maktab" so‘zini sequence o‘zgaruvchisiga yuklang.



So‘zdan unikal harflar lug‘atini (chars) yarating.<br>



Harflarni indeksga (char2idx) va indekslarni harfga (idx2char) o‘giruvchi lug‘atlarni yarating.



Kiruvchi ma’lumot (x_data) va kutilayotgan natija (y_data) uchun ro‘yxatlarni yarating. Eslab qoling, x_data oxirgi harfsiz, y_data esa birinchi harfsiz bo‘lishi kerak.



Tayyorlangan x_data va y_data ni tensorlarga aylantiring va unsqueeze() yordamida shaklini o‘zgartiring.

In [1]:
sequence = "maktab"

In [4]:
x_data = [char2idx[char] for char in sequence[:-1]]
y_data = [char2idx[char] for char in sequence[1:]]

print(f"x_data (character indices): {x_data}")
print(f"y_data (character indices): {y_data}")

x_data (character indices): [3, 0, 2, 4, 0]
y_data (character indices): [0, 2, 4, 0, 1]


In [6]:
import torch
import torch.nn as nn

# 1. Set new hyperparameters
hidden_size = 10
lr = 0.02
epochs = 200

# Re-instantiate the model with the new hidden_size
model = CharRNN(vocab_size, hidden_size, num_layers) # num_layers can remain 1 or be adjusted if needed

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Prepare data for training
x_tensor = torch.tensor(x_data).unsqueeze(0) # Add batch dimension
y_tensor = torch.tensor(y_data).unsqueeze(0) # Add batch dimension

# One-hot encode x_tensor as RNN expects input features
x_one_hot = torch.zeros(x_tensor.size(0), x_tensor.size(1), vocab_size)
x_one_hot.scatter_(2, x_tensor.unsqueeze(2), 1)

print(f"Starting training with hidden_size={hidden_size}, learning_rate={lr}, epochs={epochs}")

# 2. Training Loop
for epoch in range(epochs):
    # Initialize hidden state for each epoch
    hidden = model.init_hidden(x_tensor.size(0))

    optimizer.zero_grad() # Clear gradients

    # Forward pass
    output, hidden = model(x_one_hot, hidden)

    # Calculate loss. Reshape output for CrossEntropyLoss: (N, C, ...)
    loss = criterion(output.view(-1, vocab_size), y_tensor.view(-1))

    # Backward and optimize
    loss.backward()
    optimizer.step()

    # Print progress every 20 epochs
    if (epoch + 1) % 20 == 0:
        # Predict sequence
        _, predicted_indices = torch.max(output.data, 2) # Get predicted indices
        predicted_sequence = ''.join([idx2char[i.item()] for i in predicted_indices[0]]) # Convert to characters

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Predicted sequence: {predicted_sequence}")

print("Training complete.")

Starting training with hidden_size=10, learning_rate=0.02, epochs=200
Epoch [20/200], Loss: 0.2242, Predicted sequence: aktab
Epoch [40/200], Loss: 0.0231, Predicted sequence: aktab
Epoch [60/200], Loss: 0.0082, Predicted sequence: aktab
Epoch [80/200], Loss: 0.0053, Predicted sequence: aktab
Epoch [100/200], Loss: 0.0040, Predicted sequence: aktab
Epoch [120/200], Loss: 0.0033, Predicted sequence: aktab
Epoch [140/200], Loss: 0.0027, Predicted sequence: aktab
Epoch [160/200], Loss: 0.0023, Predicted sequence: aktab
Epoch [180/200], Loss: 0.0020, Predicted sequence: aktab
Epoch [200/200], Loss: 0.0017, Predicted sequence: aktab
Training complete.


In [5]:
import torch
import torch.nn as nn

class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size=vocab_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        # x should be one-hot encoded or embedded before passing to RNN
        # For simplicity here, we assume x is already one-hot or input feature
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)

# Instantiate the model (example)
vocab_size = len(chars)
hidden_size = 128 # You can choose any suitable hidden size
num_layers = 1 # You can choose any suitable number of layers

model = CharRNN(vocab_size, hidden_size, num_layers)
print(model)

CharRNN(
  (rnn): RNN(5, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
)


In [3]:
char2idx = {char: i for i, char in enumerate(chars)}
idx2char = {i: char for i, char in enumerate(chars)}

print(f"Character to index mapping: {char2idx}")
print(f"Index to character mapping: {idx2char}")

Character to index mapping: {'a': 0, 'b': 1, 'k': 2, 'm': 3, 't': 4}
Index to character mapping: {0: 'a', 1: 'b', 2: 'k', 3: 'm', 4: 't'}


In [2]:
chars = sorted(list(set(sequence)))
print(f"Unique characters: {chars}")

Unique characters: ['a', 'b', 'k', 'm', 't']
